## <center> **<span style="color: red">NLP</span> steps and explnation.**

Here we will use <span style="color: #ffd21e">**Hugging face**</span> trasformers pre trained models becuase it'll take so much effort to label<br>
all of our text columns "<ins>Comments, Describtion, Title</ins>" and also we will extract categorical and numirecal<br>
features from the text columns.<br><br>

##### **Steps in detail:**
<ul>
    <li>Extract categorical features from text columns sush as:<ol> 
        <li>Contains emojies
        <li>Positive or negative
        <li>language
        <li>typos</ol>
    <li> Doing analysis for most common words in comments and titles
    <li> Extracing the length of the comments, descriptions & titles
    <li> Saving those new data into new <strong>one</strong> Df to use in the final ML model

## <center> **Importing the <span style="color: red">Packeges</span>**

In [1]:
import os
import nltk
import emoji
import sqlite3
import pandas as pd
import numpy as np
import wordcloud
import transformers
import huggingface_hub

huggingface_hub.notebook_login()

## <center> **Reading the <span style="color: red">data</span>**

In [8]:
con = sqlite3.connect('../database.db')

df = pd.read_sql_query("""   
                        
                        SELECT *
                        FROM base_videos AS bv

                        INNER JOIN base_channels  AS bc ON
                            bc.channel_name = bv.channelTitle

                        INNER JOIN comments  AS c ON
                            c.video_id = bv.video_id""", con)

con.close()

df = df.T.drop_duplicates().T

In [9]:
df.head()

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,definition,duration_in_minutes,subscribers,total_views,date,video_count,about,country,comments,like_counts,reply_counts
0,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,14.835938,488000,101639693,2017-02-27 00:00:00,453,Mobile gaming content and esports organization...,US,2022 has been amazing. Can’t wait for 2023 Tri...,3,0
1,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,14.835938,488000,101639693,2017-02-27 00:00:00,453,Mobile gaming content and esports organization...,US,3:20 THAT'S WILD 💀💀💀,0,0
2,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,14.835938,488000,101639693,2017-02-27 00:00:00,453,Mobile gaming content and esports organization...,US,"I know this is a couple of weeks old, but does...",1,0
3,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,14.835938,488000,101639693,2017-02-27 00:00:00,453,Mobile gaming content and esports organization...,US,I miss Pat,0,0
4,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,14.835938,488000,101639693,2017-02-27 00:00:00,453,Mobile gaming content and esports organization...,US,I miss Pat in these videos.,1,0


##  <center> **Extract <span style="color: red">categorical</span> features**

#### *Extract emojies count per text*

In [10]:
comments_emojis_counts: list = []
title_emojis_counts: list = []
desc_emojis_counts: list = []

for comment, title, desc in zip(df["comments"], df["title"], df["description"]):
    
    comments_emojis_count: int = 0
    title_emojis_count: int = 0
    desc_emojis_count: int = 0
    
    for comment_char, title_char, desc_char in zip(comment, title, desc):
        
        if emoji.is_emoji(comment_char):
            comments_emojis_count += 1
            
        if emoji.is_emoji(title_char):
            title_emojis_count += 1
            
        if emoji.is_emoji(desc_char):
            desc_emojis_count += 1
    
    comments_emojis_counts.append(comments_emojis_count)
    title_emojis_counts.append(title_emojis_count)
    desc_emojis_counts.append(desc_emojis_count)

In [20]:
df["comments_emojis_count"] = comments_emojis_counts
df["title_emojis_count"] = title_emojis_counts
df["desc_emojis_count"] = desc_emojis_counts

df.sample(5)

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,definition,...,date,video_count,about,country,comments,like_counts,reply_counts,comments_emojis_count,title_emojis_count,desc_emojis_count
1171,orYf15jizww,Grizzley Gang Gaming,Episode 17: GG Got Me In Jail?! | GTA RP | Gri...,TEE GRIZZLEY Playing GTA RP\nEpisode 17 Of Sea...,"['tee grizzley gta 5', 'tee grizzley', 'tee gr...",2023-03-03 00:00:08+00:00,224412,2857,328,hd,...,2021-01-19 00:00:00,1306,TEE GRIZZLEY (GRIZZLEY WORLD OWNER):\nhttps://...,US,The investigation scene is hilarious 😭😭😭😂,0,0,4,0,0
189,PXilxI_iJyM,Daxua Gaming,New🔥Tips And Trick Hold The Phone Properly Whe...,A 18 Years Old Vietnam 🇻🇳 God Level Skills And...,"['pubg bgmi', 'tips and trick noob pro', 'new ...",2023-04-06 15:42:05+00:00,38674,2170,194,hd,...,2019-09-28 00:00:00,854,Follow me on Facebook: https://www.facebook.co...,IN,Op,0,1,0,0,0
552,Mi7hJuNTwzs,Happy Prince Gaming,New Evo Mp5 🤣🥕Candy Bunny Gun Skin Good Or Bad...,Free Fire New Event Video\nFree Fire Noob To P...,"['free fire', 'new mp5', 'candy bunny', 'new m...",2023-03-17 06:11:02+00:00,53929,5203,487,hd,...,2018-06-29 00:00:00,538,Exclusively Managed by : XNetwork\nBusiness Ma...,IN,Oll are noob,0,0,0,0,0
2017,c_dY_C42drk,Top Collection Gaming,Rank မတက်လို့စိတ်ညစ်‌နေလား?D‌ စောက်ရမ်းကြမ်းတဲ...,#edith #edithgameplay #edithitems #edithbestbu...,"['mlbb myanmar', 'mlbb hero guide', 'mlbb item...",2023-03-08 11:30:16+00:00,77301,4583,748,hd,...,2021-01-21 00:00:00,456,😎 စောက်ချောကြီးရဲ့ မရယ်ရတဲ့ Gameplay အလန်းစားတ...,SG,အားေနတယ်နာ်,1,0,0,0,0
410,2JI2UMupAjU,SQUEEZIE GAMING,Tout le monde a peur de lui 😨 (Among Us),Il fait des moves si imprévisibles sur Among U...,"['squeezie best of', 'squeezie moments marrant...",2023-03-30 16:00:01+00:00,1086680,64802,491,hd,...,2018-09-10 00:00:00,628,En live sur ma chaîne Twitch chaque Mercredi d...,FR,Sheshounet comment il joue mal j’ai jure 😭,0,0,0,1,0


#### *Language detection & transilation*

In [30]:
language_detector = transformers.pipeline("text-classification",
                                          model="papluca/xlm-roberta-base-language-detection",
                                          use_auth_token= True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'label': 'ur', 'score': 0.9938288331031799}

In [37]:
%%timeit
language_detector("Hello that **** code please run fatsre than that", max_length=512)[0]["label"]

1.62 s ± 85.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Looks like we will take a lot to <ins>just classify the language</ins> of each text at least: $1.5\times2000\times3$ second whitch equalls <span style="color: red">**2.5 hours**</span><br>
So we can't do translation for now becuase it at least may take **5 hours** to run so we will just use this language detetion.

In [ ]:
comments_languages = []
descs_languages = []
titels_languages = []

for comment, title, desc in zip(df["comments"][:10], df["title"][:10], df["description"][:10]):
    
    comments_languages.append(language_detector(comment, max_length=512)[0]["label"])
    descs_languages.append(language_detector(title, max_length=512)[0]["label"])
    titels_languages.append(language_detector(desc, max_length=512)[0]["label"])

In [ ]:
translated_comments

#### *Text nature classification*

In [ ]:
language_detector = pipeline("text2text-generation", model="joeddav/xlm-roberta-large-xnli")
